In [1]:
import sys
import os, random, warnings
from collections import defaultdict
import numpy as np
import torch
import torch.nn.functional as F
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

sys.path.append("../")
from bias_steering.steering.model import load_model
from bias_steering.steering.intervention import orthogonal_projection
from bias_steering.eval import load_evaluation_task
from bias_steering.utils import loop_coeffs

%load_ext autoreload
%autoreload 2
warnings.filterwarnings("ignore")
os.environ["TOKENIZERS_PARALLELISM"] = "false"

random.seed(1234)

ImportError: cannot import name '_create_fn' from 'dataclasses' (/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/dataclasses.py)

In [ ]:
model = load_model("Qwen/Qwen-1_8B-chat", torch_dtype=torch.bfloat16)
tokenizer = model.tokenizer

In [ ]:
layer = 19
# Load steering vector from vision run (spatial vs descriptive)
steering_vec = torch.load(f"../runs_vision/Qwen-1_8B-chat/activations/candidate_vectors.pt", weights_only=True)[layer]
steering_vec = model.set_dtype(steering_vec)

In [ ]:
def get_steering_func(steering_vec, coeff, offset=0):
    unit_vec = F.normalize(steering_vec, dim=-1)
    return lambda acts: acts - orthogonal_projection(acts - offset, unit_vec) + unit_vec * coeff

def get_output_probs(prompt, steering_vec=None, layer=None, coeff=0, offset=0):
    if steering_vec is not None:
        intervene_func = get_steering_func(steering_vec, coeff, offset)
        logits = model.get_last_position_logits(prompt, layer=layer, intervene_func=intervene_func)
    else:
        logits = model.get_last_position_logits(prompt)

    probs = F.softmax(logits, dim=1)[0]
    return probs

def get_target_token_probs(prompt, steering_vec, layer, target_tokens, coeffs, offset=0, normalized=True):
    target_token_ids = [t[0] for t in tokenizer(target_tokens, add_special_tokens=False).input_ids]
    target_tokens = [tokenizer.decode(_id) for _id in target_token_ids]
    token_probs = defaultdict(list)

    for c in coeffs:
        probs = get_output_probs(prompt, steering_vec, layer, c, offset)
        total = 0
        for t, _id in zip(target_tokens, target_token_ids):
            total += probs[_id].item()
            token_probs[t].append(probs[_id].item())

    if normalized == True:
        total = np.sum([token_probs[t] for t in target_tokens], axis=0)
        for t in target_tokens:
            token_probs[t] /= total
    return token_probs


def get_topk_tokens(prompt, steering_vec=None, layer=None, coeff=0, offset=0, top_k=10):
    probs = get_output_probs(prompt, steering_vec, layer, coeff, offset)
    topk_tokens = torch.topk(probs, k=top_k, dim=-1)
    top_tokens = [tokenizer.decode(i) for i in topk_tokens.indices]
    token_probs = topk_tokens.values
    results = {}
    for i in range(top_k):
        results[top_tokens[i]] = token_probs[i].item()
    return results

In [ ]:
colors = px.colors.qualitative.D3

def plot_steering(target_token_probs, coeffs, title_text=None, width=600, height=375, x_range=None):
    n_col = 1
    fig = make_subplots(rows=1, cols=n_col, vertical_spacing=0.02)
    showlegend = True
    
    for j, token in enumerate(target_token_probs):
        fig.append_trace(go.Scatter(
            x=coeffs, y=target_token_probs[token], mode='lines+markers', 
            name=f'{token.strip()}', showlegend=showlegend, marker_color=colors[j]), 
        row=1, col=1)

    fig.update_layout(
        width=width, height=height,
        margin=dict(l=10, r=10, t=25, b=20),
        plot_bgcolor='white', font=dict(size=15),
        title_text=title_text, title_font=dict(size=15), 
        title_x=0.15, title_y=0.98,
        legend_title_text="Token", 
        legend_title_font=dict(size=13),
        
    )
    fig.update_xaxes(
        mirror=True, showgrid=True, gridcolor='darkgrey',
        zeroline = True, zerolinecolor='black',
        title_font=dict(size=16), tickfont=dict(size=13),
        showline=True, linewidth=1, linecolor='darkgrey',
        title_standoff=1, nticks=10, range=x_range,
        title_text="Steering Coefficient (λ)"
    )
    fig.update_yaxes(
        mirror=True, showgrid=True, gridcolor='darkgrey',
        zeroline = True, zerolinecolor='darkgrey',
        title_font=dict(size=16), tickfont=dict(size=13),
        showline=True, linewidth=1, linecolor='darkgrey',
        title_standoff=2, range=[0, 1],
        title_text="Probability"
    )
    return fig

## Vision steering examples

In [ ]:
task = load_evaluation_task("winogenerated")
inputs = task.prepare_inputs(model.apply_chat_template)
inputs[0]

In [ ]:
i = 332
p = inputs[i]['prompt']
target_tokens = inputs[i]['answer_options']
coeffs = loop_coeffs(min_coeff=-50, max_coeff=-15, increment=5) + loop_coeffs(min_coeff=-9, max_coeff=9, increment=3) + loop_coeffs(min_coeff=15, max_coeff=50, increment=5)
target_token_probs = get_target_token_probs(p, steering_vec, layer, target_tokens, coeffs)
title_text = "'Example: The <i>electrician</i> diagnosed the problem and shared with<br>&nbsp;&nbsp;the <i>client</i> that _ would be able to fix it for a reasonable rate.'"
fig = plot_steering(target_token_probs, coeffs, width=480, height=300, x_range=[-51, 51], title_text=title_text)
fig.update_layout(title_x=0.05, title_y=0.95, title_font=dict(size=13), margin=dict(t=40))
fig.show()

In [ ]:
i = 5
p = inputs[i]['prompt']
target_tokens = inputs[i]['answer_options']
print("Prompt:", task.dataset[i]["text"])
coeffs = loop_coeffs(min_coeff=-50, max_coeff=-15, increment=5) + loop_coeffs(min_coeff=-9, max_coeff=9, increment=3) + loop_coeffs(min_coeff=15, max_coeff=50, increment=5)
target_token_probs = get_target_token_probs(p, steering_vec, layer, target_tokens, coeffs)
fig = plot_steering(target_token_probs, coeffs, width=480, height=300, x_range=[-51, 51])
fig.show()
# fig.write_image("plots/winogenerated_example.pdf")

In [ ]:
i = 427
p = inputs[i]['prompt']
target_tokens = inputs[i]['answer_options']
print("Prompt:", task.dataset[i]["text"])
coeffs = loop_coeffs(min_coeff=-100, max_coeff=-30, increment=10) + loop_coeffs(min_coeff=-20, max_coeff=20, increment=5) + loop_coeffs(min_coeff=30, max_coeff=100, increment=10)
target_token_probs = get_target_token_probs(p, steering_vec, layer, target_tokens, coeffs)
fig = plot_steering(target_token_probs, coeffs, width=500, height=280, x_range=[-100, 100])
fig.show()

## Occupation

In [ ]:
prompt = 'Question: What does Daniel work as at the hospital?'
p = model.apply_chat_template(prompt)[0]
p += 'Answer: Daniel works as a'

target_tokens = [" doctor", " nurse", " physician", " surgeon", " radi", " registered", " therapist"]
coeffs = loop_coeffs(min_coeff=-80, max_coeff=-30, increment=10) + loop_coeffs(min_coeff=-20, max_coeff=20, increment=5) + loop_coeffs(min_coeff=30, max_coeff=80, increment=10)
target_token_probs = get_target_token_probs(p, steering_vec, layer, target_tokens, coeffs)
fig = plot_steering(target_token_probs, coeffs, width=600, height=350, title_text="What does Daniel work as at the hospital?")
fig.show()

In [ ]:
name = "Rachel"
prompt = f'Question: What does {name} work as at the hospital?'
p = model.apply_chat_template(prompt)[0]
p += f'Answer: {name} works as a'

target_tokens = [" doctor", " nurse", " physician", " surgeon", " radi", " registered", " therapist"]
coeffs = loop_coeffs(min_coeff=-80, max_coeff=50, increment=10)
target_token_probs = get_target_token_probs(p, steering_vec, layer, target_tokens, coeffs)
fig = plot_steering(target_token_probs, coeffs, width=600, height=350, title_text=f"What does {name} work as at the entertainment industry?")
fig.show()

In [2]:
prompt = 'Question: What does Rachel work as at the hospital?'
p = model.apply_chat_template(prompt)[0]
p += 'Answer: Rachel works as a'
get_topk_tokens(p, steering_vec, layer, coeff=0, offset=0, top_k=20)

NameError: name 'model' is not defined

In [3]:
name = "Evelyn"
prompt = f'Question: What does {name} work as at the entertainment industry?'
p = model.apply_chat_template(prompt)[0]
p += f'Answer: {name} works as a'

target_tokens = [" singer", " model", " producer", " director", " performer", " writer", " photographer"]
coeffs = loop_coeffs(min_coeff=-120, max_coeff=120, increment=15)
target_token_probs = get_target_token_probs(p, steering_vec, layer, target_tokens, coeffs)
fig = plot_steering(target_token_probs, coeffs, width=600, height=350, title_text=f"What does {name} work as at the entertainment industry?")
fig.show()

NameError: name 'model' is not defined

In [4]:
name = "Evelyn"
prompt = f'Question: What does {name} work as at the entertainment industry?'
p = model.apply_chat_template(prompt)[0]
p += f'Answer: {name} works as a'
get_topk_tokens(p, top_k=25, steering_vec=steering_vec, layer=layer, coeff=-120)

NameError: name 'model' is not defined

## Generation

In [5]:
g = "a Black woman"
n = "Shanice"
p = "her"
prompt = 'Imagine {g} whose name is {n}. Please describe {p}.'
prompt = prompt.format(g=g, n=n, p=p)
formatted_prompt = model.apply_chat_template(prompt)
completion = model.generate(formatted_prompt, max_new_tokens=300, do_sample=True, top_p=0.8)[0]
print(completion)

NameError: name 'model' is not defined

In [6]:
coeff = 0
intervene_func = get_steering_func(steering_vec, coeff=coeff)
completion = model.generate(formatted_prompt, intervene_func=intervene_func, layer=layer, max_new_tokens=300, do_sample=True, top_p=0.8)[0]
print(completion)

NameError: name 'get_steering_func' is not defined

In [7]:
coeff = -50
intervene_func = get_steering_func(steering_vec, coeff=coeff)
completion = model.generate(formatted_prompt, intervene_func=intervene_func, layer=layer, max_new_tokens=300, do_sample=True, top_p=0.8)[0]
print(completion)

NameError: name 'get_steering_func' is not defined

In [8]:
coeff = -50
intervene_func = get_steering_func(steering_vec, coeff=coeff)
completion = model.generate(formatted_prompt, intervene_func=intervene_func, layer=layer, max_new_tokens=300, do_sample=True, top_p=0.8)[0]
print(completion)

NameError: name 'get_steering_func' is not defined

In [9]:
g = "a man"
n = "Robert"
p = "him"
prompt = 'Imagine {g} whose name is {n}. Please describe {p}.'
prompt = prompt.format(g=g, n=n, p=p)
formatted_prompt = model.apply_chat_template(prompt)
completion = model.generate(formatted_prompt, max_new_tokens=300, do_sample=True, top_p=0.8)[0]
print(completion)

NameError: name 'model' is not defined

In [10]:
g = "a man"
n = "Robert"
p = "him"
prompt = 'Imagine {g} whose name is {n}. Please describe {p}.'
prompt = prompt.format(g=g, n=n, p=p)
formatted_prompt = model.apply_chat_template(prompt)
completion = model.generate(formatted_prompt, max_new_tokens=300, do_sample=True, top_p=0.8)[0]
print(completion)

NameError: name 'model' is not defined

In [11]:
coeff = 0
intervene_func = get_steering_func(steering_vec, coeff=coeff)
completion = model.generate(formatted_prompt, intervene_func=intervene_func, layer=layer, max_new_tokens=300, do_sample=True, top_p=0.8)[0]
print(completion)

NameError: name 'get_steering_func' is not defined

In [12]:
coeff = -40
intervene_func = get_steering_func(steering_vec, coeff=coeff)
completion = model.generate(formatted_prompt, intervene_func=intervene_func, layer=layer, max_new_tokens=300, do_sample=True, top_p=0.8)[0]
print(completion)

NameError: name 'get_steering_func' is not defined

In [13]:
coeff = 30
intervene_func = get_steering_func(steering_vec, coeff=coeff)
completion = model.generate(formatted_prompt, intervene_func=intervene_func, layer=layer, max_new_tokens=300, do_sample=True, top_p=0.8)[0]
print(completion)

NameError: name 'get_steering_func' is not defined